# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
def build_classifiers():
    
    # fill this part
    
    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)

    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)

    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)

    return neighbors, linear_regression, qda

    #return None # and here

In [4]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier(); # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.7


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [9]:
def set_new_weights(old_weights, alpha, model):
    # fill the code here (two lines)
    new_weights = old_weights * np.exp(np.multiply(alpha,calculate_accuracy_vector(model.predict(test_set), test_labels)))
    Zt = np.sum(new_weights)
    return new_weights / Zt

def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

def calculate_error(weights, model):
    predicted = model.predict(test_set)
    return np.dot(weights,calculate_accuracy_vector(predicted, test_labels))


Train the classifier with the code below:

In [10]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    error_rate = calculate_error(weights, model)
    alpha = np.log((1-error_rate)/error_rate)
    weights = set_new_weights(weights, alpha, model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00094518 0.00094518 0.00106314 0.00106314 0.00094518 0.00106314
 0.00106314 0.00094518 0.00094518 0.00094518 0.00106314 0.00094518
 0.00106314 0.00094518 0.00106314 0.00094518 0.00094518 0.00094518
 0.00094518 0.00106314 0.00106314 0.00094518 0.00106314 0.00094518
 0.00106314 0.00106314 0.00094518 0.00106314 0.00106314 0.00106314
 0.00106314 0.00094518 0.00106314 0.00094518 0.00094518 0.00094518
 0.00106314 0.00094518 0.00094518 0.00094518 0.00094518 0.00106314
 0.00094518 0.00106314 0.00094518 0.00106314 0.00106314 0.00106314
 0.00094518 0.00094518 0.00106314 0.00094518 0.00106314 0.00106314
 0.00106314 0.00106314 0.00094518 0.00094518 0.00106314 0.00106314
 0.00094518 0.00094518 0.00094518 0.00106314 0.00094518 0.00094518
 0.00106314 0.00106314 0.00106314 0.00106314 0.00106314 0.00094518
 0.00094518 0.00094518 0.00087873 0.00094518 0.00106314 0.00106314
 0.00106314 0.00094518 0.00094518 0.00106314 0.00094518 0.00094518
 0.00094518 0.00106314 0.00094518 0.00094518 0.00094518 0.0010

Set the validation data set:

In [15]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [16]:
def get_prediction(x):
    # fill the code here (5-6 lines)
    predictions = []
    for i in range(len(classifiers)):
        predicted = classifiers[i].predict(x)
        predictions.append(predicted)
    return np.sign(np.sum(np.dot(alphas,predictions)))

Test it:

In [17]:
prediction = get_prediction(validate_x)[0]

print(prediction)

ValueError: shapes (0,) and (10,1) not aligned: 0 (dim 0) != 10 (dim 0)